## This script adds the following properties

#### To the 4 quadrant `Atom` instances:
- 'Arb C' = int: Arbitrary number 0 or 1 associated with alkene carbon atoms

- 'Sterimol' = dict: Made up of following keys: 'b1','b5,'l','bfs{bfs_limit}
Note: {bfs_limit} is defined early in file, default is 2
Note2: This represents the the Sterimol calculation of the entire unit

'Arb C Con' = int: Index of Arbitrary atom the atom is connected to

#### To the whole `Molecule`

- 'Arb C{i}' = int: index of the atom that is the arbitrary atom
  - Note: {i} is 0 or 1 depending on if it's arbitrary atom 0 or 1

- 'Arb C{i} Con' = tuple: the two atom indexes connected to arbitrary atom
  - Note: {i} is 0 or 1 depending on if it's arbitrary atom 0 or 1

- 'All Arb Sterimol' = dict: dict of dicts with 4 keys. 
  - These keys are the strings of the four indexes, that are connected to the alkene carbon, and they lead to the Sterimol dict made up of 'b1','b5,'l','bfs{bfs_limit}

In [1]:
import molli as ml
import numpy as np
from collections import deque
from tqdm import tqdm
import math
import networkx as nx

def angle(v1,v2):
    
    uv1 = uv(v1)
    uv2 = uv(v2)
    return np.arccos(np.clip(np.dot(uv1, uv2), -1.0, 1.0))*180/math.pi

def origin_coords(ml_mol:ml.Molecule,i: int):
    v = np.array(-1*ml_mol.coords[i])
    # new_coords = 
    return ml_mol.coords + v[np.newaxis, :]

def kabsch_rotation_matrix(P, Q, center= True):
    """Construct Kabsch rotation matrix.

    Constructs the rotation matrix that overlays the points in P with the points in Q
    with minimum RMSD. https://en.wikipedia.org/wiki/Kabsch_algorithm

    Args:
        P: Coordinates to be rotated
        Q: Reference coordinates
        center: Whether to center P and Q at origin.

    Returns:
        R: Rotation matrix
    """
    P = np.array(P)
    Q = np.array(Q)

    # Calculate centroids and center coordinates
    if center:
        centroid_P = np.mean(P, axis=0)
        centroid_Q = np.mean(Q, axis=0)
        P -= centroid_P
        Q -= centroid_Q

    # Calculate cross-covariance matrix
    H = P.T @ Q

    # Perform SVD
    U, S, V_T = np.linalg.svd(H)

    # Determine correction for right-hand coordinate system
    d = np.sign(np.linalg.det(V_T.T @ U.T))

    # Obtain rotation matrix
    R = V_T.T @ np.array([[1, 0, 0], [0, 1, 0], [0, 0, d]]) @ U.T

    return R

def uv(vec):
    return vec / np.linalg.norm(vec)

def morfeus_sterimol(
        sub: ml.Substructure,
        a1: ml.Atom,
        a2: ml.Atom,
        l_correct=True,
        return_vec=False):
    '''This is a mixture of a few different implementations. Taken from the 
    following two sources:

    wSterimol
    10.1021/acscatal.8b04043
    https://github.com/bobbypaton/wSterimol

    Morfeus
    https://github.com/digital-chemistry-laboratory/morfeus

    The L-correction adds on the difference between the bond length and vdw radius 
    of the second atom, and is by default on as this is how it was calculated in Verloop's
    original paper.

    Parameters
    ----------
    sub : ml.Substructure
        Substructure to calculate Sterimol On (includes Attachment Point)
    a1 : ml.Atom
        Dummy Atom/Attachment Point
    a2 : ml.Atom
        Atom to begin calculating Sterimol From
         
    l_correct : bool, optional
        Use Original Correction defined by Sterimol, by default True

    Returns
    -------
    float | np.ndarray
        This will return either 3 floats corresponding to B1, B5, and L,
        or the vectors defined by B1, B5, and L
    '''
    all_coords = origin_coords(sub, sub.get_atom_index(a2))

    #The original vector and unit vector connecting a1 (attachment point) and a2 (first real atom)
    v_a12 = sub.vector(a1,a2)
    bond_length = np.linalg.norm(v_a12)
    uv_a12 = np.divide(v_a12, bond_length)

    #This rotates the projection vector L into the x axis to result in consistent behavior.
    x_axis = np.array([[1.0, 0.0, 0.0]])
    R = kabsch_rotation_matrix(uv_a12.reshape(1,-1), x_axis,center=False)
    all_coords = (R @ all_coords.T).T


    no_dummy_coords = np.vstack([x for i,x in enumerate(all_coords) if i != sub.get_atom_index(a1)])
    no_dummy_vdw = np.vstack([x.vdw_radius for i,x in enumerate(sub.atoms) if i != sub.get_atom_index(a1)])

    # New vector after rotation, has same magnitude as original vector, just different direction
    fix_v_a12 = np.subtract(all_coords[sub.get_atom_index(a2)],all_coords[sub.get_atom_index(a1)])
    fix_uv_a12 = np.divide(fix_v_a12, bond_length).reshape(1,-1)

    # raise ValueError()

    unextended_l_vals = np.dot(fix_uv_a12,no_dummy_coords.T)
    proj_l_vals = np.add(unextended_l_vals, no_dummy_vdw.T)
    # print(proj_l_vals)
    # raise ValueError()

    l = np.max(proj_l_vals) + bond_length

    # A bit weird, but seems like original sterimol adds on the difference between the bond length and vdw radius of atom B. For a C-H bond this is 1.50 - 1.10 = 0.40 Angstrom)
    if l_correct:
        l += 0.4

    # Get rotation vectors in yz plane (not too sure why there are this many rotation vectors (i.e. 3600))
    r = 1
    theta = np.linspace(0, 2 * math.pi, 3600)
    x = np.zeros(len(theta))
    y = r * np.cos(theta)
    z = r * np.sin(theta)
    rot_vectors = np.column_stack((x, y, z))

    unextended_b_vals = np.dot(rot_vectors, no_dummy_coords.T)

    proj_b_vals = np.add(unextended_b_vals, no_dummy_vdw.T)

    b_vals = np.max(proj_b_vals, axis=1)

    b1 = np.min(b_vals)
    b5 = np.max(b_vals)
    if return_vec:
        l_vec = fix_uv_a12 * l

        l_vec = l_vec.reshape(-1)

        b1_vec = rot_vectors[np.argmin(b_vals)]*b1

        b5_vec = rot_vectors[np.argmax(b_vals)]*b5
        return b1_vec,b5_vec,l_vec
    else:
        return b1,b5,l

def vdw_sphere(r: float):
    '''Returns volume of a sphere

    Parameters
    ----------
    r : float
        radius of sphere
    '''
    return 4/3 * math.pi * r**3

def fast_vdw_vol(ml_mol: ml.Molecule, correct_h=False):
    '''This implements the fast calculation of vdW volume as described in:
    
    J. Org. Chem. 2003, 68, 19, 7368–7373
    DOI: 10.1021/jo034808o

    Equation
    --------
    V_vdW = sum(all atom vdW volumes) - (5.92 * N_b) - (14.7 * R_A) - (3.8 * R_NR)

    - N_b = # of bonds
    - R_A = # of aromatic rings
    - R_NR = # of non-aromatic rings

    

    Parameters
    ----------
    ml_mol : ml.Molecule
        Molecule to run volume calculation for
    correct_h : bool, optional
        The original disclosure uses a bondi radius of 1.2 angstroms instead of 1.1. Molli utilizes the radii shown
        in 10.1021/jp8111556 and 10.1021/j100785a001, which should be correct. The original uncorrected version can be
        accessed with this parameter.
    '''

    if correct_h:
        #Finds the vdw volumes for every atom and sums them
        vdw_vols = sum([vdw_sphere(a.vdw_radius) for a in ml_mol.atoms])
    else:
        #Uncorrected bondi radius equation
        vdw_vols = sum([vdw_sphere(a.vdw_radius) if a.element != ml.Element.H else vdw_sphere(1.2) for a in ml_mol.atoms])

    N_b = ml_mol.n_bonds
    R_A = 0
    R_NR = 0

    #Converts the molli molecule into an NetworkX object
    nx_mol = ml_mol.to_nxgraph()

    #This iterates through the basis cycles (i.e. the linearly independent rings) and finds aromatic and non-aromatic rings
    for ra_subgraph in nx.cycle_basis(nx_mol):
        atom_ar_types = [a.atype == ml.AtomType.Aromatic for a in ra_subgraph]
        bond_ar_types = [ml_mol.lookup_bond(a1, a2).btype == ml.BondType.Aromatic for a1, a2 in nx.find_cycle(nx_mol, ra_subgraph)]
        if all(atom_ar_types) & all(bond_ar_types):
            R_A += 1
        else:
            R_NR += 1

    #Calculates the final volume
    V_vdW = vdw_vols - (5.92 * N_b) - (14.7 * R_A) - (3.8 * R_NR)

    # print(f'V_vdW={V_vdW}\nvdw_vols = {vdw_vols}\nN_b = {N_b}\nR_A = {R_A}\n R_NR = {R_NR}')
    return V_vdW

def yield_bfsd(ml_mol: ml.Molecule, start:ml.Atom, first:ml.Atom, done: list):
    '''
    Built from Molli 1.0 molli.chem.bond implementation.
    This will return a generator object
    '''
    _sa = start
    visited = set((_sa, *done))
    q = deque()
    if first is None:
        q.append((start,0))
    else:
        visited.add(first)
        q.append((first,1))
        yield (first, 1)
    while q:
        start, dist = q.pop()
        for a in ml_mol.connected_atoms(start):
            if a not in visited:
                yield (a, dist+1)
                visited.add(a)
                q.appendleft((a, dist + 1))

def bfsd_sub(
        ml_mol: ml.Molecule,
        visited_atoms:list,
        ap: ml.Atom,
        a1:ml.Atom,
        limit: int | None
)-> ml.Substructure:
    '''This creates a full substructure or breadth-first-search (BFS) substructure 
    based on whether a limit is present. The atoms to be avoided should be provided
    and an attachment point

    Parameters
    ----------
    ml_mol : ml.Molecule
        Molecule to Traverse
    visited_atoms : list
        Atoms that should be avoided within the BFS
    ap : ml.Atom
        Atom attachment point to be appended to the list
    limit : int | None
        This is the BFS-limit, should start at 0

    Returns
    -------
    ml.Substructure
    '''
    bfsd = list(yield_bfsd(m,ap,a1,visited_atoms))
    final_bfsd = list()
    for (a,dist) in bfsd:
        if limit:
            if dist <= limit:
                final_bfsd.append(a)
        else:
            final_bfsd.append(a)

    final_bfsd.append(ap)

    return ml.Substructure(ml_mol,final_bfsd)

def correct_sub(sub: ml.Substructure, alkene_atoms) -> ml.Molecule:
    '''Removes the alkene atom and associated bonds from the substructure for volume calculation later

    Parameters
    ----------
    sub : ml.Substructure
        Substructure of alkene
    alkene_atoms : list
        Current alkene atoms in the full molecule

    Returns
    -------
    ml.Molecule
        Copy of Molecule Substructure with alkene atom removed
    '''
    #Creates property to recognize upon creation of new molecule from substructure
    for a in alkene_atoms:
        if a in sub.atoms:
            a.attrib['Remove'] = 1
    #Creates new substructure and removes carbons associated with alkene carbon
    new_mol = ml.Molecule(sub, name=sub._parent.name)
    for a in new_mol.atoms:
        if 'Remove' in a.attrib.keys():
            new_mol.del_atom(a)

    return new_mol

In [2]:
#The number of atoms starts at 0, so "2" refers to 3-atom breadth-first search
bfs_limit = 2

mlib = ml.MoleculeLibrary('5_1_DB_mols_OPT.mlib')
mlib_ster = ml.MoleculeLibrary('5_2_DB_OPT_Sterimol.mlib', readonly=False, overwrite=True)

with mlib.reading(), mlib_ster.writing():
    for k in tqdm(mlib):
        m = mlib[k]
        alk_bool = np.array([True if v == "1" else False for v in m.attrib['_Alkene_w_H']])

        a_arr = np.array(m.atoms)

        alk_a = a_arr[alk_bool]

        connect = {}

        #This iterates through the two arbitrary carbons of the alkene c0 and c1 and assigns arbitrary but consistent labels
        desc = {}
        for i,a in enumerate(alk_a):
            a: ml.Atom
            a.attrib[f'Arb C'] = i
            m.attrib[f'Arb C{i}'] = m.get_atom_index(a)
            _con_atoms = [x for x in m.connected_atoms(a) if x not in alk_a]
            m.attrib[f'Arb C{i} Con'] = tuple(m.get_atom_index(x) for x in _con_atoms)
            # print([m.get_atom_index(x) for x in _con_atoms])
            #Iterates through the atoms connected to c0 and c1 and assigns the arbitrary but consistent labels
            for j,atom in enumerate(_con_atoms):

                full_sub = bfsd_sub(
                    ml_mol=m,
                    visited_atoms=alk_a,
                    ap=a,
                    a1=atom,
                    limit=None
                )

                b1, b5, l = morfeus_sterimol(
                    full_sub,
                    a1=a, # Attachment point (i.e. c0/c1)
                    a2=atom, # Fragment
                    l_correct=True, #Original Implementation
                    return_vec=False
                )

                new_sub_mol = correct_sub(full_sub, alk_a)
                full_vol = fast_vdw_vol(new_sub_mol, correct_h=True)

                bfs_sub = bfsd_sub(
                    ml_mol=m,
                    visited_atoms=alk_a,
                    ap=a,
                    a1=atom,
                    limit=bfs_limit
                )

                bfs_b1,bfs_b5,bfs_l = morfeus_sterimol(
                    bfs_sub,
                    a1=a, # Attachment point (i.e. c0/c1)
                    a2=atom, # Fragment
                    l_correct=True, #Original Implementation
                    return_vec=False 
                )

                new_bfs_sub_mol = correct_sub(bfs_sub, alk_a)
                bfs_vol = fast_vdw_vol(new_bfs_sub_mol, correct_h=True)

                #Assigns descriptors to atom and molecule
                _desc = {'b1':b1,'b5':b5,'l':l,'vol': full_vol, f'bfs{bfs_limit}':bfs_vol}
                atom.attrib['Sterimol'] = _desc
                atom.attrib[f'Arb C Con'] = m.get_atom_index(a)
                
                # Must be string key due to msgpack strict_map_key 
                # (may be unnecessary in future implementations)
                desc[f'{m.get_atom_index(atom)}'] = {'b1':b1,'b5':b5,'l':l,'vol': full_vol, f'bfs{bfs_limit}_vol':bfs_vol}
        
        m.attrib[f'All Arb Sterimol'] = desc

        mlib_ster[k] = m

100%|██████████| 784/784 [00:18<00:00, 41.62it/s]
